In [ ]:
import os
import json
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Đường dẫn đến thư mục dữ liệu
dataset_path = "/kaggle/input/btxrd-data/BTXRD/"
json_dir = os.path.join(dataset_path, "Annotations")
image_dir = os.path.join(dataset_path, "images")

# Danh sách các nhãn ung thư ác tính
malignant_labels = [
    "osteosarcoma", "ewing's sarcoma", "chondrosarcoma", "fibrosarcoma",
    "malignant", "metastatic", "multiple myeloma", "giant cell tumor", 
    "dedifferentiated chondrosarcoma"
]

# Lấy danh sách tất cả file JSON
json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

# Chọn ngẫu nhiên một file JSON
random_json = random.choice(json_files)
json_path = os.path.join(json_dir, random_json)

# Đọc dữ liệu từ JSON
with open(json_path, "r") as file:
    data = json.load(file)

# Kiểm tra ảnh tương ứng
if "imagePath" in data:
    image_path = os.path.join(image_dir, data["imagePath"])
    
    if os.path.exists(image_path):
        image = cv2.imread(image_path)
        height, width, _ = image.shape
        total_pixels = height * width  # Tổng số pixel ảnh
    else:
        print(f"⚠ Ảnh {data['imagePath']} không tồn tại!")
        total_pixels = 0
else:
    print("⚠ Không tìm thấy đường dẫn ảnh trong file JSON!")
    total_pixels = 0

# Tính toán diện tích vùng ung thư & xác định loại ung thư
tumor_area = 0
tumor_type = "Benign (Lành tính)"  # Mặc định là lành tính

if "shapes" in data:
    for shape in data["shapes"]:
        label = shape.get("label", "").lower()  # Chuyển nhãn về chữ thường
        points = np.array(shape["points"], dtype=np.int32)

        if len(points) > 2:  # Chỉ tính nếu đủ điểm để tạo hình đa giác
            tumor_area += cv2.contourArea(points)

        # Kiểm tra xem nhãn có trong danh sách ác tính không
        if any(malignant in label for malignant in malignant_labels):
            tumor_type = "Malignant (Ác tính)"  # Nếu có bất kỳ nhãn nào trùng, chuyển thành ác tính

# Tính tỷ lệ vùng ung thư
tumor_ratio = (tumor_area / total_pixels) * 100 if total_pixels > 0 else 0

# Hiển thị ảnh với vùng annotation
if os.path.exists(image_path):
    img = cv2.imread(image_path)
    for shape in data["shapes"]:
        points = np.array(shape["points"], dtype=np.int32)
        cv2.polylines(img, [points], isClosed=True, color=(0, 255, 0), thickness=2)

    plt.figure(figsize=(6, 6))
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f"{data['imagePath']} - {tumor_type}\nTumor Ratio: {tumor_ratio:.2f}%")
    plt.axis("off")
    plt.show()

# Xuất kết quả
print(f"📌 Ảnh: {data['imagePath']}")
print(f"📌 Tổng số pixel: {total_pixels}")
print(f"📌 Tổng diện tích vùng ung thư: {tumor_area}")
print(f"📌 Tỷ lệ vùng ung thư: {tumor_ratio:.2f}%")
print(f"📌 Kết luận: {tumor_type}")
